# Azure AI Foundry: Models, AI Search, AI Document Intelligence, Agent Service

## Setup

In [ ]:
import json
from dataclasses import dataclass

import requests

# ai document intelligence
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, AnalyzeResult
from azure.ai.inference import ChatCompletionsClient, EmbeddingsClient
from azure.ai.inference.prompts import PromptTemplate

# ai foundary
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AzureAISearchTool,
    AzureFunctionStorageQueue,
    AzureFunctionTool,
    BingGroundingTool,
    CodeInterpreterTool,
    ConnectionType,
    FileSearchTool,
)

# azure
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# ai search
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

## Authentication

In [ ]:
!az login

In [ ]:
credential = DefaultAzureCredential()

## Connection

In [ ]:
# project connection
project_connection_string = (
    "eastus2.api.azureml.ms;XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)
project = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=credential
)

In [ ]:
# List the properties of all connections
connections = project.connections.list()
print(f"Listing of all connections ({len(connections)}):")
for connection in connections:
    print(connection)

In [ ]:
# get endpoints and api keys
connection_names = []
connections = project.connections.list()
for each_connection in connections:
    connection_names.append(each_connection.name)
connection_endpoints_keys = []
for each_connection_name in connection_names:
    try:
        each_connection = project.connections.get(
            connection_name=each_connection_name,
            include_credentials=True,
        )
        each_output = {
            "name": each_connection.name,
            "endpoint": each_connection.endpoint_url,
            "connection_type": each_connection.connection_type,
            "key": each_connection.key,
        }
        connection_endpoints_keys.append(each_output)
    except Exception as e:
        pass
connection_endpoints_keys

## Embeddings

In [ ]:
# embeddings
embeddings = project.inference.get_embeddings_client(
    connection_name="XXXXXXXXXXXXXXXXXXXXXXXXXX",
)
emb = embeddings.embed(input="testing 123", model="text-embedding-3-small")
print(emb.data[0].embedding[0:10])

## Inference 

In [ ]:
# auth to model endpoint via API key
# also can use credential=DefaultAzureCredential(), credential_scopes=["https://cognitiveservices.azure.com/.default"]
# azure openai api-version: https://aka.ms/azsdk/azure-ai-inference/azure-openai-api-versions
api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
api_version = "2024-06-01"
chat_client = ChatCompletionsClient(
    endpoint="https://XXXXXXXXXXXXXXXXXXXXXXXX.openai.azure.com/openai/deployments/gpt-4o-mini",
    credential=AzureKeyCredential(api_key),
    api_version=api_version,
)
response = chat_client.complete(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful question and answer agent"},
        {"role": "user", "content": "Who was the lead singer of Van Halen?"},
    ],
    temperature=0.9,
    max_tokens=100,
)
print(response.choices[0].message.content)

In [ ]:
# auth via azure ai project api
chat = project.inference.get_chat_completions_client(
    connection_name="XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)
response = chat.complete(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful question and answer agent"},
        {"role": "user", "content": "Who was the lead singer of Van Halen?"},
    ],
    temperature=0.5,
    max_tokens=100,
)
print(response.choices[0].message.content)

In [ ]:
# prompt templates
prompt_template = PromptTemplate.from_string(
    prompt_template="""
    system:
    You are a helpful question and answer agent.
    The user's first name is {{first_name}} and their last name is {{last_name}}.
    user:
    Tell me about the band Van Halen
    """
)
# generate system message from the template, passing in the context as variables
messages = prompt_template.create_messages(first_name="John", last_name="Smith")
print(messages, end="\n\n")
response = chat.complete(
    messages=messages,
    model="gpt-4o-mini",
    **prompt_template.parameters,
)
print(response.choices[0].message.content)

## AI Search

In [ ]:
# ai search connection via ai foundry
search_connection = project.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, include_credentials=True
)
print(search_connection.name)
print(search_connection.endpoint_url)
# index
index_client = SearchIndexClient(
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key),
)
search_client = SearchClient(
    index_name="hotels-sample-index",
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key),
)

In [ ]:
# search
response = search_client.search("hotels in the united states", top=3)
for each_index, each in enumerate(response):
    print(f"Result #{each_index+1}")
    print(each, end="\n\n")

## AI Document Intelligence

In [ ]:
# connection params
ai_document_intelligence_endpoint = (
    "https://aifoundaryhubXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/"
)
ai_document_intelligence_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# connection
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=ai_document_intelligence_endpoint,
    credential=AzureKeyCredential(ai_document_intelligence_key),
)

In [ ]:
# sample document
formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
# can also use local pdf files:
# import base64
# with open(file_path, "rb") as f:
#    base64_encoded_pdf = base64.b64encode(f.read()).decode("utf-8")
#    analyze_request = {
#        "base64Source": base64_encoded_pdf
#    }
poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl)
)
result = poller.result()

In [ ]:
# processing
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )
for page in result.pages:
    for line_idx, line in enumerate(page.lines):
        print(
            "...Line # {} has text content '{}'".format(
                line_idx, line.content.encode("utf-8")
            )
        )
    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' and has a confidence of {}".format(
                selection_mark.state, selection_mark.confidence
            )
        )
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
            table_idx, table.row_count, table.column_count
        )
    )
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content.encode("utf-8"),
            )
        )

## Other AI Services 

In [ ]:
# computer vision: dense captioning
services_endpoint = "https://aifoundaryhubXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/"
endpoint_url = f"{services_endpoint}computervision/imageanalysis:analyze?features=denseCaptions&gender-neutral-caption=false&api-version=2023-10-01"
services_api_key = (
    "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)
headers = {
    "Ocp-Apim-Subscription-Key": services_api_key,
    "Content-Type": "application/json; charset=utf-8",
}
image_url = (
    "https://ai.azure.com/common/vision/denseCaptioning/DenseCaptioningSample2.png"
)
json_data = {"uri": image_url}
response = requests.post(endpoint_url, headers=headers, json=json_data)
response_content = response.text
data = json.loads(response_content)
print(data)

## Agent Service

### Create Agent

In [ ]:
# create agent
code_interpreter = CodeInterpreterTool()
agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="my-agent",
    instructions="""
    You are helpful agent that can generate and run Python code
    Instructions:
    1. talk like a pirate
    2. be nice 
    """,
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)
print(f"Created agent, agent ID: {agent.id}")
# create a thread
# a thread stores the back-and-forth messages between user and agent
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

### Messages

In [ ]:
# create a message
message = project.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Create a bar chart for the operating profit using the following data and provide the file to me? 
    Company A: $1.2 million
    Company B: $2.5 million
    Company C: $3.0 million
    Company D: $1.8 million
    """,
)
print(f"Created message, message ID: {message.id}")
# run agent
run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(10)
    run = project.agents.get_run(thread_id=thread_id, run_id=run.id)

In [ ]:
# get messages
messages = project.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

### Delete Agent

In [ ]:
# delete agent
project.agents.delete_agent(agent.id)
print(f"Deleted agent: {agent.name}")

### Get Agent

In [ ]:
# get created agent
project.agents.get_agent(agent_id="AGENT_ID")

### Pre-Built Agent Templates

In [ ]:
# file search agent
uploaded_ids = []
for fp in files:
    upl = project.agents.upload_file_and_poll(file_path=fp, purpose="assistants")
    uploaded_ids.append(upl.id)
vs = project.agents.create_vector_store_and_poll(
    file_ids=uploaded_ids, name="vector_store_name"
)
file_search_tool = FileSearchTool(vector_store_ids=[vstore_id])
agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="search-agent",
    instructions="""
        Instructions:
        1. Answer user questions
        1. search the provided files
    """,
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# grounding agent
bing_connection = project.connections.get(connection_name="bing_connection_name")
conn_id = bing_connection.id
bing_tool = BingGroundingTool(connection_id=conn_id)
agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="bing-agent",
    instructions="""
        Instructions:
        1. Answer user questions
        2. Use Bing for real-time references.
    """,
    tools=bing_tool.definitions,
    headers={"x-ms-enable-preview": "true"},
)
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# ai search agent
all_connections = project.connections.list()
for c in all_connections:
    if c.connection_type == ConnectionType.AZURE_AI_SEARCH:
        conn_id = c.id
ai_search_tool = AzureAISearchTool(index_connection_id=conn_id, index_name="INDEX_NAME")
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="agent-search",
    instructions="""
    Answer user questions, look up documents via AI search
    """,
    tools=ai_search_tool.definitions,
    tool_resources=ai_search_tool.resources,
    headers={"x-ms-enable-preview": "true"},
)
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# azure functions
storage_endpoint = "STORAGE_ENDPOINT"
azure_function_tool = AzureFunctionTool(
    name="foo",
    description="Get advice from 'Foo'.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "The question to ask Foo."},
            "outputqueueuri": {
                "type": "string",
                "description": "The output queue URI.",
            },
        },
    },
    input_queue=AzureFunctionStorageQueue(
        queue_name="azure-function-foo-input",
        storage_service_endpoint=storage_endpoint,
    ),
    output_queue=AzureFunctionStorageQueue(
        queue_name="azure-function-tool-output",
        storage_service_endpoint=storage_endpoint,
    ),
)
agent = project.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="azure-function-agent-foo",
    instructions="""
    You are a helpful support agent.
    If the user says 'What would foo say?' then call the foo function
    """,
    tools=azure_function_tool.definitions,
)
print(f"Created agent, agent ID: {agent.id}")